Import packages to make this a lot easier

In [7]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3



EXTRACTING THE RAW DATA USING WEB API, as json

In [8]:
# taken from documentation
DATABASE_LOCATION = "sqlite://my_played_tracks.sqlite"
# my spotify username
USER_ID = "bc0303"
# must be regenerated every 24 hours
TOKEN = "BQBFgKNhyUXIpE2G8xEPD7ItICAiyqN_Vz70Zx-eEJycpIsOvo0dUD073DFWPxYXmpezOMswI65AqA7icAtIxQ2QL1Z54xYNmqr6t4voQNhDsFD-5kCDHV8YbeTY7QnKiOVmzRI2o6s3hS4"

# spotify listening history object
if __name__ == "__main__":
    headers = {"Accept": "application/json", "Content-Type": "application/json",
               "Authorization": "Bearer {token}".format(token=TOKEN)}
# declaring variables that will be useful to me!
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)
yesterday_unix_timestamp = int(yesterday.timestamp())*1000
# request 
r = requests.get("https://api.spotify.com/v1/me/player/recently-played?limit=10&after={time}".format(
    time=yesterday_unix_timestamp), headers=headers)
data = r.json()
# Look!
print(data)

{'error': {'status': 401, 'message': 'The access token expired'}}


TRANSFORMING THE DATA INTO READABLE VERSION

In [9]:
# Extracting only the relevant bits of data from the json object      

song_names = []
artist_names = []
played_at_list = []
timestamps = []

# make it look pretty!
for song in data["items"]:
    song_names.append(song["track"]["name"])
    artist_names.append(song["track"]["album"]["artists"][0]["name"])
    played_at_list.append(song["played_at"])
    timestamps.append(song["played_at"][0:10])
        
# Prepare a dictionary in order to turn it into a pandas dataframe below       
song_dict = {
     "song_name" : song_names,
      "artist_name": artist_names,
     "played_at" : played_at_list,
     "timestamp" : timestamps
  }

song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])
print(song_df)

KeyError: 'items'

VALIDATING THE DATA TO SEE THAT WE HAVE NO DUPLICATES OR ENCOUNTER SCENARIOS WE DON'T LIKE

In [ ]:
def check_if_valid_data(df: pd.DataFrame) -> bool:
    # Check if dataframe is empty
    if df.empty:
        print("No songs downloaded. Finishing execution")
        return False 

    # Primary Key Check
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("Primary Key check is violated")

    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null values found")

    # Check that all timestamps are of yesterday's date
    yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)

    timestamps = df["timestamp"].tolist()
    for timestamp in timestamps:
        if datetime.datetime.strptime(timestamp, '%Y-%m-%d') != yesterday:
            raise Exception("At least one of the returned songs does not have a yesterday's timestamp")

    return True

RUNNING THE VALIDATION FUNCTION

In [ ]:
# Validate
if check_if_valid_data(song_df):
    print("Data valid, proceed to Load stage")

LOADING THE DATA INTO A SQL DATABASE

In [ ]:
DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"
engine = sqlalchemy.create_engine(DATABASE_LOCATION)
conn = sqlite3.connect('my_played_tracks.sqlite')
cursor = conn.cursor()

sql_query = """
    CREATE TABLE IF NOT EXISTS my_played_tracks(
        song_name VARCHAR(200),
        artist_name VARCHAR(200),
        played_at VARCHAR(200),
        timestamp VARCHAR(200),
        CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
    )
    """

cursor.execute(sql_query)
print("Opened database successfully")

try:
        song_df.to_sql("my_played_tracks", engine, index=False, if_exists='append')
except:
        print("Data already exists in the database")

conn.close()
print("Close database successfully")